<a href="https://colab.research.google.com/github/dbsdlsdud/AIStudy24/blob/master/%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D_%EA%B7%B8%EB%A6%AC%EB%93%9C%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* 지금까지 배운 알고리즘은 훈련세트와 테스트세트를 나누어 평가를 진행하는데
* 결국 테스트세트로 평가를하면 테스트세트에 맞는 모델이 만들어지는 결론이 나온다.
* 테스트세트에 일반화 성능을 올바르게 예측하려면 가능한 테스트세트를 사용하지 말아야 한다.
* 테스트세트는 모델을 만들고 마지막에 딱 한 번 사용하는 것이 좋다.

결정트리에서 max_depth 매개변수를 사용한 하이퍼파라미터 튜닝해보기

* 테스트세트를 사용하지 않으려면 모델이 과대적합인지 과소적합인지 판단하기 어렵다.
* 테스트세트를 사용하지 않고 측정하는 간단한 방법은 훈련세트를 또 나누는 것이다.
* 훈련세트 60%, 검증세트 20%, 테스트세트 20% -> 실무에서 많이 쓰는 기법

-

* 테스트하고 싶은 매개변수를 바꿔가면서 가장 좋은 모델이 나올 수 있다.
* 이 매개변수를 사용해 훈련세트와 검증세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련
* 훈련세트에서 모델을 훈련하고 검증세트로 모델을 평가
* 마지막 세트에서 테스트세트를 이용해 최종점수 평가

In [1]:
# 검증세트
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split
# 훈련세트, 테스트세트 나누기
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state=42) # 20%
# 원본데이터 : 6497개 0.2(20%) = train(5197), test(1300)

# 검증세트 20% 생성
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

print(sub_input.shape, val_input.shape)
# 원래 5197개였던 훈련세트가 4157개로 줄고, 검증세트가 1040개 됐다.

(4157, 3) (1040, 3)


In [2]:
# sub_input, sub_target과 val_input, val_target을 사용해 모델을 만들고 평가
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target) # 훈련

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))
# 과대적합(0.9971133028626413) 매개변수를 바궈서 더 좋은 모델을 찾아야 됨

0.9971133028626413
0.864423076923077


* 검증세트를 만드느라 훈련세트가 줄었다.(이럴때 많은 데이터를 훈련하면 좋은결과가 나옴)
* 교차검증 : 검증세트를 떼어 내 평가하는 과정을 여러번 반복
* [훈련세트, 훈련세트, 검증세트]
* [훈련세트, 검증세트, 훈련세트]
* [검증세트, 훈련세트, 훈련세트] -> 패리티 방식(3 - 폴드 교차 검증)
* 5 - 폴드, 10 - 폴드도 있음
* 사이킷런에는 cross_validate()라는 교차 검증 함수를 사용한다.

In [3]:
from sklearn.model_selection import cross_validate
# 평가할 모델 객체를  첫번재 매개변수로 전달(직접 검증세트를 떼지않고 훈련세트 전체를 전달)
scores = cross_validate(dt, train_input, train_target)
# scores = cross_validate(dt, train_input, train_target, cv = 5)
# cv 매개변수를 이용해 폴드 수를 변경 cv=5(기본값)
print(scores)
# fit_time(훈련시간), score_time(검증시간), test_score(최종점수) 키를 가진 딕셔너리(k : v)를 반환

{'fit_time': array([0.01994395, 0.01642656, 0.01263261, 0.02027345, 0.01780987]), 'score_time': array([0.00190115, 0.00184011, 0.00173354, 0.00740981, 0.00173283]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [4]:
import numpy as np
print(np.mean(scores['test_score'])) # test_score 이름이지만 검증폴드의 점수다.
# 5개의 검증 점수를 평균을 내어 최종 점수를 얻는다. 85.53%

0.855300214703487


* 주의 : cross_validate()는 훈련세트를 섞어서 폴드를 나누지 않았다 -> 분할기 사용(splitter)
* 앞서 우리는 train_test_split()함수로 전체 데이터를 섞은 후 훈련세트를 준비했다.
* 만약 교차 검증을 할때 훈련세트를 섞으려면 분할기(splitter)를 지정해야 한다.
* 사이킷런의 분할기는 교차검증에서 폴드를 어떻게 나눌지 결정
* cross_validate()는 기본적으로 회귀모델인 KFold분할기를 사용
* 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해서 StratifiedKFold를 사용

In [5]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [6]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42) # 10 - 폴드 교차 검증 수행
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


* 결정트리의 매개변수 값을 바꿔가며 가장 좋은 성능이 나오는 모델을 찾아봐야 한다.
* 테스트세트를 사용하지 않고 교차 검증을 통해 좋은모델을 골라야 함

하이퍼파라미터 튜닝
* 모델 파라미터 : 머신러닝 모델이 학습하는 파라미터
* 하이퍼파라미터 : 사용자가 지정해야 하는 파라미터
* 하이퍼파라미터 튜닝 : 라이브러리가 제공하는 기본값을 그대로 사용해 모델을 훈련
* 그 다음 검증세트의 점수나 교차 검증을 통해서 매개변수를 조금씩 변경 1~2, 5~6개의 매개변수를 제공한다
* AutoML : 사람의 개입없이 하이퍼파라미터 튜닝을 자동으로 수행하는 기술
*Max_dapth 를 최적으로 고정하고 min_sample_split을 바꿔가며 최적의 값을 찾는다. 면 값이 함께 변경됨
* -> 두개의 매개변수를 동시에 바꿔가며 최적의 값을 찾아야 됨 -> 다중 for문을 사용해야 돼서 복잡해짐
* 사이킷런에서 제공하는 그리드 서치(Grid Search)가 제공 GridSearchCV

In [8]:
from sklearn.model_selection import GridSearchCV # 그리드서치로 크로스 검증
# min_impurity_decrease 매개변수의 최적값 찾아보기
# min_sample_split : 샘플을 최소한 몇개 이상이어야 하위노드로 분리(split) 할 것인지
# 클수록 가지치기(과대적합방지), 적을 수록 정확하기(과대적합)
params = {'min_impurity_decrease': [0.0001,0.0002,0.0003,0.0004,0.0005]} # 0.0001 ~ 증가 5번
# 매개변수와 탐색할 값을 리스트를 딕셔너리로 만듬
print(params)

{'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}


In [9]:
# params를 하나씩 5번 실행 , n_jobs = 1 : 모든 코어
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42),params, n_jobs = -1)
gs.fit(train_input, train_target) # 훈련 시작

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

gs = GridSearchCV(DecisionTreeClassifier(random_state = 42),params, n_jobs = -1)
* 결정 트리 클래스의 객체를 생성하자마자 바로 전달
* 일반 모델을 훈련하는 것 처럼 fit() 메서드를 호출
* 이 메서드를 호출하면 서치 객체는 결정트리 모델 min_impurity_decrease 값을 바꿔가면서 총 5번 수행
* GridSearchCV의 cv매개변수 기본 값 : 5 / 5*5 = 25번 수행
* n_jobs에서 병렬 실행에 사용될 CPU 코어수를 지정(-1로 사용시 모든 코어)

gs.fit(train_input, train_target) # 훈련 시작
* 교차 검증에서 초적의 하이퍼파라미터를 찾으면 전체 훈련 세트로 모델을 다시 만들어야 하지만
* 사이킷런의 그리드 서치는 검증 점수가 가장 높은 모델의 매개변수 조합으로
* 전체 훈련 세트에서 자동으로 다시 모델을 훈련 한다.

-
* 이렇게 최적화되어 훈련된 모델은 GridSearchCV클래스로 만들어진
* 객체의 best_estimator_ 속성에 저장되어 있고
* 최적의 매개변수는 best_params_ 속성에 저장되어 있다.

In [13]:
print(f'그리드 서치를 통해 찾은 최적의 모델 정확도: {dt.score(train_input, train_target)*100:.2f}%')
print(f'최적의 매개변수 : {gs.best_params_}')
# 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_속성의 'mean_test_score'키에 저장되어 있다.
print('5번의 교차 검증으로 얻은 점수 : ',gs.cv_results_['mean_test_score']) # 가장 큰 값 : 첫번째

그리드 서치를 통해 찾은 최적의 모델 정확도: 97.06%
최적의 매개변수 : {'min_impurity_decrease': 0.0001}
5번의 교차 검증으로 얻은 점수 :  [0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


* 눈으로 보는 거 보다 넘파이를 이용해 argmax()함수를 사용하면 가장 큰 값의 인덱스를 추출할 수 있다.
* 그 다음 이 인덱스를 사용해 params키에 저장된 매개변수 출력해보기
* 이 값이 최상의 검증 점수를 만든 매개변수의 조합이 된다.

In [17]:
best_index = np.argmax(gs.cv_results_["mean_test_score"]) # 0.86819297
print(gs.cv_results_['params'][best_index])
# 앞에서 출력한 best_params_과 같은지 확인
print(f'beat_params : {gs.best_params_}')

{'min_impurity_decrease': 0.0001}
beat_params : {'min_impurity_decrease': 0.0001}
0


과정 요약
1. 탐색할 매개변수 지정
2. 훈련세트에서 그리드 서치 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾아서 그리드 서치에 저장
3. 그리드 서치는 최상의 매개변수에서 전체 훈련세트를 사용해 최종 모델을 훈련하고 그리드 서치에 저장

* 조금 더 복잡한 매개변수 조합 해보기
* 노드를 분할하기 위한 불순도 감소 최소량 지정 max_depth(트리의 깊이)
* min_samples_eplit노드를 나누기 위한 최소 샘플 수
* 넘파이 arange() 함수는 첫번째 매개변수 값에서 시작해서 두번째 매개변수에 도달할 때까지 세번째 매개변수를 계속 더한 배열을 만듬
* 0.0001 ~ 0.001까지 0.0001을 더한 배열(두번째 매개변수는 포함되지 않음) = 9번
* 파이썬 ragne() 함수는 정수만 사용가능 5~20까지 1씩 증가 = 15개
* 교차 검증 횟수 9 * 15 * 10 = 1350개 * 5 - 폴드교차 = 6750개

In [20]:
params = {'min_impurity_decrease':np.arange(0.0001,0.001,0.0001), # 9번
          'max_depth':range(5,20,1),                              # 5~20번까지 15번
          'min_samples_split':range(2,100,10)}  # 분할 할 수 있는 샘플 수 지정  2~100번까지 10번
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target) # 훈련 시작

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [21]:
print(gs.best_params_) # 최상의 매개변수 조합확인

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [22]:
print(np.max(gs.cv_results_['mean_test_score'])) # 최상의 교차 점수 확인
# 개선점 : 탐색할 매개변수 간격을 0.0001 혹은 1로 설정 했는데 근거 부족(좁히거나 넓힐 필요성)

0.8683865773302731


* 매개변수 값이 수치일 때 값의 범위나 간격을 미리 정하기가 어려울 수 있음
* 너무 많은 매개 변수 조건이 있어 그리드 서치 수행시간이 오래 걸린다
* 해결 : 랜덤 서치 사용
* 랜덤서치 : 매개변수 값이 목록을 전달하는 것이 아니라 매개변수를 샘플링 할 수 있는 확률 분포도 객체를 전달
* 싸이파이 : 파이썬의 핵심 과학 라이브러리로 적분, 보간, 선형대수, 확률 등 포함한 수치 계산용 전용 라이브러리(코랩에 내장)

In [23]:
from scipy.stats import uniform, randint
# uniform, randint 클래스는 모두 주어진 범이에서 고르게 값을 뽑는다( 균등 분포에서 샘플링)
# randint()에서는 정수 값을 추출, uniform()은 실수 값을 뽑음

# 정수용 테스트
rgen = randint(0,10) # 10개의 숫자를 샘플링
rgen.rvs(10) # 난수 발생기(랜덤과 유사)
# 샘플링 숫자를 늘리면 쉽게 확인 가능

array([2, 2, 2, 9, 8, 2, 5, 3, 5, 4])

In [27]:
np.unique(rgen.rvs(1000), return_counts=True) # 1000개를 샘플링해서 각 숫자의 개수를 세어봄

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 84, 115,  98, 100,  96,  87, 109, 103, 114,  94]))

In [28]:
# 실수용 테스트
ugen = uniform(0,1) # 10개의 숫자를 샘플링
ugen.rvs(10) # 난수 발생기(랜덤과 유사)

array([0.45651587, 0.54485631, 0.66123495, 0.19657634, 0.87874112,
       0.15244022, 0.7633066 , 0.30395587, 0.92712977, 0.18478506])

In [29]:
params = {'min_impurity_decrease':uniform(0.0001,0.001), # 0.0001 ~ 0.001의 실수값
          'max_depth':randint(20,50),                    # 20~50 사이의 정수
          'min_samples_split':randint(2,25),             # 2~25 사이의 정수
          'min_samples_leaf':randint(1,25),}             # 1~25 사이의 정수
          # 리프 노드가 되기 위한 최소 샘플 갯수(지식 노드의 샘플 수가 이 값보다 작으면)

In [31]:
# 샘플링 횟수는 사이킷런의 랜덤서치 클래스인 RandomizedSearchCV의 n_iter매개변수에 저장
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,n_iter=100,n_jobs=-1,random_state=42)
# n_iter=100 총 100번을 샘플링하여 교차 검증을 수행(최적의 매개변수 조합을 찾음)
gs.fit(train_input,train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7bb2cf76ba30>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7bb2cf769ed0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7bb2cf76b580>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7bb2cf76b820>},
                   random_state=42)

In [32]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [36]:
print(np.max(gs.cv_results_['mean_test_score'])) # 최고의 교차 검증 점수 확인
dt = gs.best_estimator_
# 최종 모델로 결정하고 테스트 세트 성능 확인
print(dt.score(test_input, test_target))

0.8695428296438884
0.86


0.86
